In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
week15_link = 'http://www.footballlocks.com/nfl_odds_week_15.shtml#Closing%20NFL%20Odds%20Week%201,%202007'

#get html text
week15_html = requests.get(week15_link).text

soup = BeautifulSoup(week15_html, 'html.parser')

#want just table row data
tr = soup.find_all("tr")

len(tr)

#start of table
print(tr[51])

#last entry in table
print(tr[276])

<tr>
<td>12/14 8:25 ET</td>
<td>Denver</td>
<td>-3</td>
<td>At Indianapolis</td>
<td>40.5</td>
<td>-$145 +$125</td>
</tr>
<tr>
<td>12/18 8:30 ET</td>
<td>At Indianapolis</td>
<td>-3.5</td>
<td>Cincinnati</td>
<td>54.5</td>
<td>-$175 +$155</td>
</tr>


In [3]:
#want to 277 because it does not include last one
table = tr[51:277]

type(table[-1])

type(table)

string_table = [str(i) for i in table]

type(string_table[0])

week15_df = pd.DataFrame(string_table)

week15_df.columns = ['all_data']

week15_df.head()

,all_data
0,<tr>\n<td>12/14 8:25 ET</td>\n<td>Denver</td>\...
1,<tr>\n<td>12/16 4:30 ET</td>\n<td>At Detroit</...
2,<tr>\n<td>12/16 8:25 ET</td>\n<td>At Kansas Ci...
3,<tr>\n<td>12/17 1:00 ET</td>\n<td>Philadelphia...
4,<tr>\n<td>12/17 1:00 ET</td>\n<td>At Carolina<...


In [4]:
#want to clean the date data

dates = [x.split('td')[1] for x in week15_df['all_data']]

cleaned_dates = []
for x in dates:
    str1 = x.replace(">", "").replace("<", "")
    final_str = str1[:-1]
    cleaned_dates.append(final_str)

week15_df['date'] = cleaned_dates

In [5]:
#pull out favorites

week15_df['cleaned_favorites'] = week15_df['all_data'].map(lambda x: x.split('td')[3].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create underdog column

week15_df['cleaned_underdogs'] = week15_df['all_data'].map(lambda x: x.split('td')[7].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create spread column
week15_df['cleaned_spread'] = week15_df['all_data'].map(lambda x: x.split('td')[5].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create over_under column
week15_df['cleaned_over_under'] = week15_df['all_data'].map(lambda x: x.split('td')[9].replace('<', '').replace('>', '').replace('/', '')
                                                 if len(x.split('td'))>3 else np.nan)

#create money_odds column and clean data
money_odds = []
for x in week15_df['all_data']:
    try:
        money_odds.append(x.split('td')[11].replace('<', '').replace('>', '').replace('/', ''))
    except:
        money_odds.append(np.nan)
        
week15_df['money_odds'] = money_odds

#input week number for each game
week15_df['week_number'] = '15'

In [6]:
#want to input year for each game. on lines we want to decrease the count by one, len(week1_df.all_data[x].split('td')) = 3
year = 2017
year_list = []
for x in week15_df['all_data']:
    if len(x.split('td')) > 3:
        year_list.append(year)
    else:
        year_list.append(year)
        year -= 1

week15_df['year'] = year_list

In [7]:
#create a final dataframe for week 15
week15_final = week15_df[['date', 'year', 'week_number', 'cleaned_favorites', 'cleaned_underdogs', 'cleaned_spread', 
                        'cleaned_over_under', 'money_odds']]
week15_final.head()

,date,year,week_number,cleaned_favorites,cleaned_underdogs,cleaned_spread,cleaned_over_under,money_odds
0,12/14 8:25 ET,2017,15,Denver,At Indianapolis,-3,40.5,-$145 +$125
1,12/16 4:30 ET,2017,15,At Detroit,Chicago,-5,44,-$245 +$205
2,12/16 8:25 ET,2017,15,At Kansas City,LA Chargers,"span title=""Pick Em. No Favorite. No Underdog...",47,-$110 -$110
3,12/17 1:00 ET,2017,15,Philadelphia,At NY Giants,-7.5,41,-$350 +$290
4,12/17 1:00 ET,2017,15,At Carolina,Green Bay,-3,47,-$150 +$130


In [8]:
#week15_final to pickle
week15_final.to_pickle('week15_final.pkl')